
# # AzureTranslateTool with AgentExecutor
# 

# This notebook demonstrates how to use the `AzureTranslateTool` in conjunction with an AgentExecutor. The agent will handle translation requests by utilizing the Azure OpenAI model and the `AzureTranslateTool`.


# ## Step 1: Import Required Libraries and Load Environment Variables
# We begin by importing the necessary libraries and loading the environment variables that include the Azure OpenAI and LangSmith API keys.


In [ ]:
# Import necessary libraries
from libs.community.langchain_community.tools.azure_ai_services.translate_tool import AzureTranslateTool
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain import hub
import os
from dotenv import load_dotenv

In [ ]:
# Load environment variables from .env
load_dotenv()

In [ ]:
# Load the prompt from LangSmith
LANGSMITH_KEY = os.getenv('LANGSMITH_KEY')
prompt = hub.pull("hwchase17/structured-chat-agent", api_key=LANGSMITH_KEY)

In [ ]:
# Instantiate the AzureTranslateTool
tools = [AzureTranslateTool()]

In [ ]:
# Initialize the Azure OpenAI model
model = AzureChatOpenAI(
    openai_api_version=os.getenv('OPENAI_API_VERSION'),  
    azure_deployment=os.getenv('COMPLETIONS_MODEL'),
    azure_endpoint=os.getenv('AZURE_ENDPOINT'),
    api_key=os.getenv('AZURE_API_KEY'),
    temperature=0.5,
    max_tokens=200,
    timeout=60,
    max_retries=10,
)

In [ ]:
# Create the agent using the model, tools, and prompt
agent = create_structured_chat_agent(model, tools, prompt)
# Set up the AgentExecutor with verbose output
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

Example 1 - Simple Translation

In [ ]:

# Input text to translate
input_text = "Hello, how are you?"# Invoke the agent with the translation request
try:
    response = agent_executor.invoke({"input": f"Can you translate the following text to te reo maori: {input_text}"})
    print("Translation result:", response)
except Exception as e:
    print(f"Error during translation: {e}")


Example 2 - Language Detection

In [ ]:
# Example: Detect the language of "Bonjour tout le monde"
query_language_detection = {'input': 'Can you detect what language this phrase is in: Bonjour tout le monde'}

# Invoke the tool to detect language
try:
    response_language_detection = agent_executor.invoke(query_language_detection)
    print("Language detection result:", response_language_detection)
except Exception as e:
    print(f"Error during language detection: {e}")


Example 3 Translate Complex Sentences

In [ ]:
# Example: Translate a more complex sentence to French
query_complex_sentence = {'input': 'Can you translate this technical sentence to French: "Artificial intelligence will revolutionize healthcare in the next decade."'}

# Invoke the tool to translate the complex sentence
try:
    response_complex_sentence = agent_executor.invoke(query_complex_sentence)
    print("Complex translation result:", response_complex_sentence)
except Exception as e:
    print(f"Error during translation: {e}")


Example 4 - Language guess + translate

In [ ]:
# Example: Guess the language and translate to English
query_language_guess_and_translate = {'input': 'Can you guess what language this is and translate it to English: "Hola, ¿cómo estás?"'}

# Invoke the tool to guess language and translate
try:
    response_language_guess_and_translate = agent_executor.invoke(query_language_guess_and_translate)
    print("Guess and Translate result:", response_language_guess_and_translate)
except Exception as e:
    print(f"Error during language guess and translation: {e}")


Example 5 - Translate to Multiple Languages

In [ ]:
# Example: Translate "Good morning" into Spanish, French, and Italian
query_multiple_translations = {'input': 'Can you translate "Good morning" into Spanish, French, and Italian?'}

# Invoke the tool to translate into multiple languages
try:
    response_multiple_translations = agent_executor.invoke(query_multiple_translations)
    print("Multiple translations result:", response_multiple_translations)
except Exception as e:
    print(f"Error during translation to multiple languages: {e}")


Example 6 - Asking for Idiomatic Expressions

In [ ]:
# Example: Translate the idiom "It's raining cats and dogs" into Spanish
query_idiom_translation = {'input': 'Can you translate the idiom "It\'s raining cats and dogs" into Spanish?'}

# Invoke the tool to translate the idiom
try:
    response_idiom_translation = agent_executor.invoke(query_idiom_translation)
    print("Idiom translation result:", response_idiom_translation)
except Exception as e:
    print(f"Error during idiom translation: {e}")


Example 7 - Asking to Translate Back and Forth

In [ ]:
# Example: Translate a phrase to German and then back to English
query_back_and_forth = {'input': 'Can you translate "The quick brown fox jumps over the lazy dog" into German, and then back to English?'}

# Invoke the tool to translate back and forth
try:
    response_back_and_forth = agent_executor.invoke(query_back_and_forth)
    print("Back-and-forth translation result:", response_back_and_forth)
except Exception as e:
    print(f"Error during back-and-forth translation: {e}")


Example 8 - Translating Phrases with Special Characters or Emojis

In [49]:
# Example: Translate a phrase with text and emojis into French
query_with_text_and_emojis = {'input': 'Can you translate this phrase into French: "I\'m happy 😊 because it\'s sunny ☀️ today."'}

# Invoke the tool to translate the phrase with text and emojis
try:
    response_with_text_and_emojis = agent_executor.invoke(query_with_text_and_emojis)
    print("Emoji translation result:", response_with_text_and_emojis)
except Exception as e:
    print(f"Error during emoji translation: {e}")



INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


{
  "action": "Final Answer",
  "action_input": "Je suis content 😊 parce qu’il fait beau ☀️ aujourd’hui."
}

> Finished chain.
Emoji translation result: {'input': 'Can you translate this phrase into French: "I\'m happy 😊 because it\'s sunny ☀️ today."', 'output': 'Je suis content 😊 parce qu’il fait beau ☀️ aujourd’hui.'}


Example 9 - Longer Paragraph Translation

In [50]:
# Example: Translate a longer paragraph into Spanish
query_paragraph_translation = {'input': 'Can you translate this paragraph to Spanish: "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed."'}

# Invoke the tool to translate a paragraph
try:
    response_paragraph_translation = agent_executor.invoke(query_paragraph_translation)
    print("Paragraph translation result:", response_paragraph_translation)
except Exception as e:
    print(f"Error during paragraph translation: {e}")




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://api.cognitive.microsofttranslator.com/translate?to=REDACTED&api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '150'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'c7ccb0ff-8d0b-11ef-9df1-ccf9e4c69b56'
    'User-Agent': 'azsdk-python-ai-translation-text/1.0.1 Python/3.12.5 (Windows-10-10.0.19045-SP0)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
    'Ocp-Apim-Subscription-Region': 'REDACTED'
A body is sent with the request


{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed.",
    "to_language": "es"
  }
}

INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 18 Oct 2024 04:45:22 GMT'
    'Content-Type': 'application/json; charset=utf-8'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-alive'
    'Access-Control-Expose-Headers': 'REDACTED'
    'X-RequestId': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'X-Metered-Usage': 'REDACTED'
    'X-MT-System': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'


El aprendizaje automático es un subconjunto de la inteligencia artificial y permite a los ordenadores aprender de los datos sin necesidad de programarlos explícitamente.

INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


{
  "action": "Final Answer",
  "action_input": "El aprendizaje automático es un subconjunto de la inteligencia artificial y permite a los ordenadores aprender de los datos sin necesidad de programarlos explícitamente."
}

> Finished chain.
Paragraph translation result: {'input': 'Can you translate this paragraph to Spanish: "Machine learning is a subset of artificial intelligence, and it allows computers to learn from data without being explicitly programmed."', 'output': 'El aprendizaje automático es un subconjunto de la inteligencia artificial y permite a los ordenadores aprender de los datos sin necesidad de programarlos explícitamente.'}


Example 10 - Asking for a Reverse Translation

In [51]:
# Example: Reverse translate a complex sentence through multiple languages (English -> German -> Spanish -> English)
query_reverse_translation_complex = {
    'input': 'Translate "The quick brown fox jumps over the lazy dog" into German, then into Spanish, and finally back to English. Let\'s see if the meaning remains the same.'
}

# Invoke the tool for complex reverse translation
try:
    response_reverse_translation_complex = agent_executor.invoke(query_reverse_translation_complex)
    print("Complex reverse translation result:", response_reverse_translation_complex)
except Exception as e:
    print(f"Error during reverse translation: {e}")





> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://api.cognitive.microsofttranslator.com/translate?to=REDACTED&api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '57'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'ca5b4c99-8d0b-11ef-9995-ccf9e4c69b56'
    'User-Agent': 'azsdk-python-ai-translation-text/1.0.1 Python/3.12.5 (Windows-10-10.0.19045-SP0)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
    'Ocp-Apim-Subscription-Region': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 18 Oct 2024 04:45:26 GMT'
    'Content-Type': 'application/json; charset=utf-8'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-al

Thought: I need to translate the sentence into German first, then into Spanish, and finally back to English to check if the meaning remains the same.

Action:
```json
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "The quick brown fox jumps over the lazy dog",
    "to_language": "de"
  }
}
```Der flinke braune Fuchs springt über den faulen Hund

INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://api.cognitive.microsofttranslator.com/translate?to=REDACTED&api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '71'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'cb184303-8d0b-11ef-8a17-ccf9e4c69b56'
    'User-Agent': 'azsdk-python-ai-translation-text/1.0.1 Python/3.12.5 (Windows-10-10.0.19045-SP0)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
    'Ocp-Apim-Subscription-Region': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 18 Oct 2024 04:45:27 GMT'
    'Content-Type': 'application/json; charset=utf-8'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-al

Action:
```json
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "Der flinke braune Fuchs springt über den faulen Hund",
    "to_language": "es"
  }
}
```El ágil zorro marrón salta sobre el perro perezoso

INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://api.cognitive.microsofttranslator.com/translate?to=REDACTED&api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '74'
    'Accept': 'application/json'
    'x-ms-client-request-id': 'cbe1c833-8d0b-11ef-9955-ccf9e4c69b56'
    'User-Agent': 'azsdk-python-ai-translation-text/1.0.1 Python/3.12.5 (Windows-10-10.0.19045-SP0)'
    'Ocp-Apim-Subscription-Key': 'REDACTED'
    'Ocp-Apim-Subscription-Region': 'REDACTED'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Date': 'Fri, 18 Oct 2024 04:45:28 GMT'
    'Content-Type': 'application/json; charset=utf-8'
    'Transfer-Encoding': 'chunked'
    'Connection': 'keep-al

Action:
```json
{
  "action": "azure_translator_tool",
  "action_input": {
    "query": "El ágil zorro marrón salta sobre el perro perezoso",
    "to_language": "en"
  }
}
```The agile brown fox jumps on the lazy dog

INFO:httpx:HTTP Request: POST https://nzcapstoneproject.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01 "HTTP/1.1 200 OK"


Action:
```json
{
  "action": "Final Answer",
  "action_input": "The original sentence 'The quick brown fox jumps over the lazy dog' translated back to English after going through German and Spanish is 'The agile brown fox jumps on the lazy dog'. The meaning remains largely the same, though there are slight variations in wording."
}
```

> Finished chain.
Complex reverse translation result: {'input': 'Translate "The quick brown fox jumps over the lazy dog" into German, then into Spanish, and finally back to English. Let\'s see if the meaning remains the same.', 'output': "The original sentence 'The quick brown fox jumps over the lazy dog' translated back to English after going through German and Spanish is 'The agile brown fox jumps on the lazy dog'. The meaning remains largely the same, though there are slight variations in wording."}
